<a href="https://colab.research.google.com/github/Hariharan1882/Supply_chain/blob/main/supply_chain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import hashlib
import time
import json


class Block:
    def __init__(self, index, timestamp, transactions, previous_hash):
        self.index = index
        self.timestamp = timestamp
        self.transactions = transactions
        self.previous_hash = previous_hash
        self.nonce = 0
        self.hash = self.calculate_hash()

    def calculate_hash(self):
        block_string = json.dumps({
            'index': self.index,
            'timestamp': self.timestamp,
            'transactions': self.transactions,
            'previous_hash': self.previous_hash,
            'nonce': self.nonce
        }, sort_keys=True).encode()
        return hashlib.sha256(block_string).hexdigest()

    def mine_block(self, difficulty):
        while self.hash[:difficulty] != "0" * difficulty:
            self.nonce += 1
            self.hash = self.calculate_hash()
        print(f"✅ Block mined: {self.hash}")


class SupplyChainBlockchain:
    def __init__(self):
        self.chain = [self.create_genesis_block()]
        self.difficulty = 2
        self.pending_transactions = []
        self.authorized_entities = {
            "M001": "Manufacturer",
            "T001": "Transporter",
            "R001": "Retailer",
        }

    def create_genesis_block(self):
        return Block(0, time.time(), ["Genesis Block: Supply Chain Record"], "0")

    def get_latest_block(self):
        return self.chain[-1]

    def mine_pending_transactions(self, entity_id):
        if entity_id not in self.authorized_entities:
            print("⛔ Unauthorized Entity. Cannot mine transactions.")
            self.pending_transactions = []
            return

        block = Block(
            len(self.chain),
            time.time(),
            self.pending_transactions,
            self.get_latest_block().hash
        )
        block.mine_block(self.difficulty)

        self.chain.append(block)
        print(f"📦 Block validated by {self.authorized_entities[entity_id]}")
        self.pending_transactions = []

    def add_product_movement(self, entity_id, product_id, from_location, to_location):
        if entity_id not in self.authorized_entities:
            print("⛔ Unauthorized entity! Only registered actors can add records.")
            return

        entity_name = self.authorized_entities[entity_id]
        self.pending_transactions.append({
            "product_id": product_id,
            "from": from_location,
            "to": to_location,
            "handled_by": entity_name,
            "timestamp": time.ctime()
        })
        print(f"✅ Movement added by {entity_name}")

    def is_chain_valid(self):
        for i in range(1, len(self.chain)):
            current_block = self.chain[i]
            previous_block = self.chain[i - 1]

            if current_block.hash != current_block.calculate_hash():
                return False
            if current_block.previous_hash != previous_block.hash:
                return False
        return True

    def print_chain(self):
        for block in self.chain:
            print(json.dumps({
                "index": block.index,
                "timestamp": block.timestamp,
                "transactions": block.transactions,
                "hash": block.hash,
                "previous_hash": block.previous_hash
            }, indent=4))
            print("-" * 40)

    def track_product(self, product_id):
        history = []
        for block in self.chain:
            for tx in block.transactions:
                if isinstance(tx, dict) and tx.get("product_id") == product_id:
                    history.append(tx)

        if history:
            print(f"\n🔍 Tracking history for Product ID: {product_id}")
            for record in history:
                print(json.dumps(record, indent=4))
        else:
            print(f"\n⚠️ No movement history found for Product ID: {product_id}")


# --- CLI Demo ---
if __name__ == "__main__":
    supply_chain = SupplyChainBlockchain()

    while True:
        print("\n=== Supply Chain Product Tracking Blockchain ===")
        print("1. Add product movement (Authorized only)")
        print("2. Mine block (Validate transactions)")
        print("3. Show blockchain")
        print("4. Validate blockchain")
        print("5. Track product by ID")
        print("6. Exit")

        choice = input("Choose option: ")

        if choice == "1":
            entity_id = input("Enter Entity ID: ")
            product_id = input("Product ID: ")
            from_loc = input("From Location: ")
            to_loc = input("To Location: ")
            supply_chain.add_product_movement(entity_id, product_id, from_loc, to_loc)

        elif choice == "2":
            entity_id = input("Enter Entity ID to validate block: ")
            supply_chain.mine_pending_transactions(entity_id)

        elif choice == "3":
            supply_chain.print_chain()

        elif choice == "4":
            print("Blockchain valid?", supply_chain.is_chain_valid())

        elif choice == "5":
            product_id = input("Enter Product ID: ")
            supply_chain.track_product(product_id)

        elif choice == "6":
            break

        else:
            print("Invalid option. Try again.")



=== Supply Chain Product Tracking Blockchain ===
1. Add product movement (Authorized only)
2. Mine block (Validate transactions)
3. Show blockchain
4. Validate blockchain
5. Track product by ID
6. Exit
Choose option: 1
Enter Entity ID: M001
Product ID: P123
From Location: Factory
To Location: Warehouse
✅ Movement added by Manufacturer

=== Supply Chain Product Tracking Blockchain ===
1. Add product movement (Authorized only)
2. Mine block (Validate transactions)
3. Show blockchain
4. Validate blockchain
5. Track product by ID
6. Exit
Choose option: 2
Enter Entity ID to validate block: M001
✅ Block mined: 0077cd301b202b689664840517a4556d41db712d222eb8e095df66106590f2e1
📦 Block validated by Manufacturer

=== Supply Chain Product Tracking Blockchain ===
1. Add product movement (Authorized only)
2. Mine block (Validate transactions)
3. Show blockchain
4. Validate blockchain
5. Track product by ID
6. Exit
Choose option: 3
{
    "index": 0,
    "timestamp": 1755847466.4381146,
    "transact